In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### **Step 1**: calculation of the target "DaysOpen"

In [ ]:
df_par = pd.read_csv(
    "parto.csv",
    usecols = [
        "idAnimale", 
        "codiceIstat",
        "siglaProvincia",
        "codiceSpecieAIA",
        "codiceRazzaAIA", 
        "DataParto"
    ]
)

print(f"Initial dataset records: {df_par.shape[0]}")

df_par["DataParto"] = pd.to_datetime(df_par["DataParto"], yearfirst = True)
df_par["DataInseminazione"] = df_par["DataParto"].apply(lambda x: x - timedelta(days = 285)) # 285 days: average bovine geastation
df_par["DataInseminazione"] = pd.to_datetime(df_par["DataInseminazione"], yearfirst = True)

df_par = df_par.sort_values(by = ["idAnimale", "DataInseminazione"])
df_par["DaysOpen"] = df_par.groupby("idAnimale")["DataParto"].shift(1).sub(df_par["DataInseminazione"]).dt.days.abs()

df_par.reset_index(inplace = True, drop = True)
df_par.dropna(inplace = True)

df_par.to_csv("days_open.csv", index = False)
print(f"Final dataset records: {df_par.shape[0]}")

### **Step 2**: calculation of the age in months "EtaMesiInseminazione"

In [ ]:
df_an = pd.read_csv("anagrafica.csv", usecols = ["idAnimale", "DataNascita"])
print(f"Initial anagrafica dataset records: {df_an.shape[0]}")

df_do = pd.read_csv("days_open.csv")
print(f"Initial parto dataset records: {df_do.shape[0]}")

df = pd.merge(
    df_an.drop_duplicates(),
    df_do,
    on = ["idAnimale"],
    how = "inner"
)

df["DataNascita"] = pd.to_datetime(df["DataNascita"])
df["DataInseminazione"] = pd.to_datetime(df["DataInseminazione"])

df["EtaMesiInseminazione"] = (df["DataInseminazione"].dt.year - df["DataNascita"].dt.year) * 12 + df["DataInseminazione"].dt.month - df["DataNascita"].dt.month
df.dropna(inplace = True)

df.to_csv("days_open.csv", index = False)
print(f"Final dataset records: {df.shape[0]}")

### **Step 3**: select only "Frisona" in codiceRazzaAIA

In [ ]:
df["codiceRazzaAIA"].unique()

In [ ]:
df.replace(
    {
        "2" : "02",
        2.0 : "02", 
        "2.0" : "02"   
    }, 
    inplace = True
)

In [ ]:
df["codiceRazzaAIA"].value_counts()

In [ ]:
print(f"Frisona accounts for {round((2810748 / 3743870) * 100, 2)} % of data")

In [ ]:
df = df[df["codiceRazzaAIA"] == "02"]
df.shape

In [ ]:
sns.histplot(
    df["idAnimale"].value_counts().values,
    binwidth = 1
)

plt.axvline(x = df["idAnimale"].value_counts().values.mean(), c = "red", linestyle = "--")
plt.text(x = df["idAnimale"].value_counts().values.mean() + 0.2, y = 700000, s = "mean = 1.83", fontsize = 8, fontweight = "bold")
plt.show()

In [ ]:
idAnimal_counts = df["idAnimal"].value_counts()
idAnimal_to_retain = idAnimal_counts[idAnimal_counts <= 3].index # select only idAnimal counts <= 3

df = df[df["idAnimal"].isin(idAnimal_to_retain)]

In [ ]:
df.to_csv("days_open_frisona.csv", index = False)
print(f"Final dataset records: {df.shape[0]}")

### **Step 4**: concatenation of THI data up to 180 days

In [ ]:
df_clima = pd.read_csv("clima.csv")
print(f"Initial clima dataset records: {df_clima.shape[0]}")

df_clima_grouped = df_clima.groupby(["siglaProvincia", "DataRilevamentoClimatico"]).agg(
    {
        "ThiMinimo" : "mean",
        "ThiMassimo" : "mean"
    }
)

df_clima_grouped = df_clima_grouped.reset_index()
df_clima_grouped["DataRilevamentoClimatico"] = pd.to_datetime(df_clima_grouped["DataRilevamentoClimatico"])
df_clima_grouped["Year"] = df_clima_grouped["DataRilevamentoClimatico"].dt.year
df_clima_grouped["Month"] = df_clima_grouped["DataRilevamentoClimatico"].dt.month
df_clima_grouped["Day"] = df_clima_grouped["DataRilevamentoClimatico"].dt.day

df_do = pd.read_csv("days_open_frisona.csv")
print(f"Initial days open dataset records: {df_do.shape[0]}")

df_do["DataInseminazione"] = pd.to_datetime(df_do["DataInseminazione"])
df_do["Year"] = df_do["DataInseminazione"].dt.year
df_do["Month"] = df_do["DataInseminazione"].dt.month
df_do["Day"] = df_do["DataInseminazione"].dt.day

df = pd.merge(
    df_clima_grouped,
    df_do,
    on = ["siglaProvincia", "Year", "Month", "Day"]
)

df.rename(
    columns = {
        "ThiMinimo" : "ThiMin_dayless_0",
        "ThiMassimo" : "ThiMax_dayless_0",
    },
    inplace = True
)

for day in range(1, 181):

    if day == 1:
        df["DataRilevamentoClimatico"] = df["DataInseminazione"] - pd.Timedelta(days = day)

        df_temp = pd.merge(
            df,
            df_clima_grouped[["siglaProvincia", "DataRilevamentoClimatico", "ThiMinimo", "ThiMassimo"]],
            how = "inner",
            on = ["siglaProvincia", "DataRilevamentoClimatico"]
        )

        df_temp.rename(
            columns = {
                "ThiMinimo" : f"ThiMin_dayless_{day}",
                "ThiMassimo" : f"ThiMax_dayless_{day}",
            },
            inplace = True
        )

        df_final = df_temp
    
    else:
        df_final["DataRilevamentoClimatico"] = df_final["DataInseminazione"] - pd.Timedelta(days = day)

        df_temp = pd.merge(
            df_final,
            df_clima_grouped[["siglaProvincia", "DataRilevamentoClimatico", "ThiMinimo", "ThiMassimo"]],
            how = "inner",
            on = ["siglaProvincia", "DataRilevamentoClimatico"]
        )

        df_temp.rename(
            columns = {
                "ThiMinimo" : f"ThiMin_dayless_{day}",
                "ThiMassimo" : f"ThiMax_dayless_{day}",
            },
            inplace = True
        )

        df_final = df_temp
        
df_final.dropna(inplace = True)
df_final.to_csv("days_open_clima.csv", index = False)
print(f"Final dataset records: {df_final.shape[0]}")

### **Step 5**: selection THImax effect until 5, 15, 30, 45, 60, 75, 90, 120, 150 and 180 days from DataInseminazione (three days windows mean)

In [ ]:
df_final_120 = df_final[["idAnimale", "siglaProvincia", "EtaMesiInseminazione", "DataInseminazione", "DaysOpen"]]

day_less_to_choose = [5, 15, 30, 45, 60, 75, 90, 120, 150, 180]
new_columns = []

for number in day_less_to_choose:
    prev_col = f"ThiMax_dayless_{number - 1}"
    curr_col = f"ThiMax_dayless_{number}"
    next_col = f"ThiMax_dayless_{number + 1}"

    if prev_col in df_final.columns and curr_col in df_final.columns and next_col in df_final.columns:
        new_column_name = f"ThiMax_dayless_{number}"
        df_final_120[new_column_name] = df_final[[prev_col, curr_col, next_col]].mean(axis = 1)
        new_columns.append(new_column_name)
        
    else:
        df_final_120[f"ThiMax_dayless_{number}"] = df_final[curr_col]
        new_columns.append(f"ThiMax_dayless_{number}")

df_final_120.to_csv("days_open_clima_grouped.csv", index = False)
print(f"Final dataset records: {df_final_120.shape[0]}")

### **Step 6**: concatenation of milk quantity and quality

In [ ]:
df_lat = pd.read_csv("lattazione.csv")
print(f"Initial lattazione dataset records: {df_lat.shape[0]}")

In [ ]:
df_lat = df_lat[(df_lat["LatteEffettivo"] != "LatteEffettivo") | (df_lat["GrassoEffettivo"] != "GrassoEffettivo") | (df_lat["ProteineEffettive"] != "ProteineEffettive")] 
df_lat.shape[0]

In [ ]:
df_lat["LatteEffettivo"] = pd.to_numeric(df_lat["LatteEffettivo"])
df_lat["GrassoEffettivo"] = pd.to_numeric(df_lat["GrassoEffettivo"])
df_lat["ProteineEffettive"] = pd.to_numeric(df_lat["ProteineEffettive"])

In [ ]:
df_lat_grouped = df_lat.groupby(["siglaProvincia", "DataLattazione"]).agg(
    {
        "LatteEffettivo" : "mean",
        "GrassoEffettivo" : "mean",
        "ProteineEffettive" : "mean"
    }
)

df_lat_grouped = df_lat_grouped.reset_index()
df_lat_grouped["DataLattazione"] = pd.to_datetime(df_lat_grouped["DataLattazione"])
df_lat_grouped["Year"] = df_lat_grouped["DataLattazione"].dt.year
df_lat_grouped["Month"] = df_lat_grouped["DataLattazione"].dt.month
df_lat_grouped["Day"] = df_lat_grouped["DataLattazione"].dt.day

df_final_120["DataInseminazione"] = pd.to_datetime(df_final_120["DataInseminazione"])
df_final_120["Year"] = df_final_120["DataInseminazione"].dt.year
df_final_120["Month"] = df_final_120["DataInseminazione"].dt.month
df_final_120["Day"] = df_final_120["DataInseminazione"].dt.day

df = pd.merge(
    df_lat_grouped,
    df_final_120,
    on = ["siglaProvincia", "Year", "Month", "Day"]
)

df.dropna(inplace = True)

In [ ]:
df.to_csv("days_open_clima_grouped_lattazione.csv", index = False)
print(f"Final dataset records: {df.shape[0]}")

### **Step 7**: Data analysis

In [ ]:
df = pd.read_csv("days_open_clima_grouped_lattazione.csv")

In [ ]:
sns.set_theme(style = "ticks")

x = df["DaysOpen"]

f, (ax_box, ax_hist) = plt.subplots(
    2,
    sharex = True,
    gridspec_kw = {"height_ratios": (.15, .85)}
)

sns.boxplot(
    x,
    ax = ax_box,
    orient = "h",
    color = "lightblue",
    width = 0.5,
    fliersize = 2.0
).set_title("Distribution of target days open", fontsize = 11, fontweight = "bold")

sns.histplot(
    x,
    ax = ax_hist,
    bins = 50,
    kde = True,
    alpha = 0.5
)

ax_box.set(yticks = [])
sns.despine(ax = ax_hist)
sns.despine(ax = ax_box, left = True)

plt.xlabel("Days open values", fontsize = 11)
plt.ylabel("Counts", fontsize = 11)
plt.xticks(np.arange(0, 2001, step = 500), fontsize = 9)
plt.yticks(np.arange(0, 900001, step = 100000), fontsize = 9)
plt.axvline(x = 1.322523e+02, c = "red", linestyle = "--")
plt.text(x = 1.402523e+02, y = 900000, s = "mean", fontsize = 8, fontweight = "bold")
plt.show()

In [ ]:
sns.stripplot(
    df["DaysOpen"],
    orient = "h",
    color = "lightblue"
).set_title("Distribution of target days open", fontsize = 11, fontweight = "bold")

plt.xlabel("")
plt.show()

In [ ]:
plt.figure(figsize = (20, 8))

sns.boxplot(
    df.iloc[:, 7:15],
    orient = "h",
    width = 0.5,
    fliersize = 2.0    
)

plt.show()

In [ ]:
days_less = [5, 15, 30, 45, 60, 75, 90, 120]
upper_bound_list = []

for day_less in days_less:
    Q1 = df[f"ThiMax_dayless_{day_less}"].quantile(0.25)
    Q3 = df[f"ThiMax_dayless_{day_less}"].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    upper_bound_list.append(upper_bound)
    
    df_out = df[df[f"ThiMax_dayless_{day_less}"] > upper_bound]
    
    print(f"ThiMax_dayless_{day_less}: {lower_bound, upper_bound}")
    
    plt.figure(figsize = (20, 5))
    sns.barplot(x = df_out["idCountry"].value_counts().index, y = df_out["idCountry"].value_counts().values)
    plt.show()
    
print(np.mean(upper_bound_list))

In [ ]:
# Remove extreme values on THI

ts = np.mean(upper_bound_list)
df_filtered = df[~((df[['ThiMax_dayless_5', 'ThiMax_dayless_15', 'ThiMax_dayless_30', 'ThiMax_dayless_45', 'ThiMax_dayless_60', 'ThiMax_dayless_75', 'ThiMax_dayless_90', "ThiMax_dayless_120"]] > ts).any(axis = 1))]
df_filtered.shape

In [ ]:
df_filtered.iloc[:, 7:15].describe()

In [ ]:
sns.histplot(
    df_filtered["DaysOpen"],
    bins = 50,
    kde = True,
    alpha = 0.5
)

plt.show()

In [ ]:
df_filtered = df_filtered[df_filtered["DaysOpen"] >= 21] # remove DaysOpen values under 21
df_filtered["DaysOpen"].describe()

In [ ]:
df_filtered = df[df["AgeMonthsInsemination"] > 20] # removed AgeMonthsInsemination values under 20
df_filtered.shape

In [ ]:
df_filtered.reset_index(inplace = True, drop = True)

In [ ]:
df.iloc[:, 7:16].corr()

In [ ]:
df[["AgeMonthsInsemination", "gPFT", "Milk", "Fat", "Proteins", "DaysOpen"]].corr()

In [ ]:
df_filtered.to_csv("df_ready.csv", index = False)
print(f"Final dataset records: {df.shape[0]}")

### **Step 8**: Final dataset

In [ ]:
df = pd.read_csv("df_ready.csv")

In [ ]:
df = df[
    [
        "idAnimale",
        "siglaProvincia",
        "EtaMesiInseminazione",
        "LatteEffettivo",
        "GrassoEffettivo",
        "ProteineEffettive",
        "ThiMax_dayless_5",
        "ThiMax_dayless_15",
        "ThiMax_dayless_30",
        "ThiMax_dayless_45",
        "ThiMax_dayless_60",
        "ThiMax_dayless_75",
        "ThiMax_dayless_90",
        "ThiMax_dayless_120",
        "DaysOpen"
    ]
]

In [ ]:
df.rename(
    columns = {
        "idAnimale" : "idAnimal",
        "siglaProvincia" : "idCountry",
        "EtaMesiInseminazione" : "AgeMonthsInsemination",
        "LatteEffettivo" : "Milk",
        "GrassoEffettivo" : "Fat",
        "ProteineEffettive" : "Proteins",
    },
    inplace = True
)

In [ ]:
df.to_csv("df_ready.csv", index = False)